 # Modelling tests Machine Learning

je teste ici un modèle SVM basé sur les attributs calculés sur le signal



### Table of Contents

* [datasets](#datasets)
    * [creation de la base](#dataset_1)
* [SVM sur features](#svm_features)
    * [base sous echantillonée](#svm_features_sous)
        * [test](#svm_features_sous_test)
        * [gridsearch](#svm_features_sous_grid)
     * [base sur echantillonée](#svm_features_sur)
 * [Resultats](#svm_features_sur_res)


In [153]:
import os
import sys
sys.path.append('../src')

from balance import load_balanced
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## datasets <a class="anchor" id="datasets"></a>

tests de models sur:
 * la base sous-echantillonée
 * la base sur-echantillonée

### creation de la base <a class="anchor" id="dataset_1"></a>

sauvegarde des signaux des bases dans un .csv dans raw/processed ainsi qu'un csv contenant les features calculées

base sous echantillonée N signaux par classe = 641

In [24]:
if not os.path.isfile('../data/processed/mitbih_train_under.csv'):
    Xs,ys = load_balanced.load_balanced_data(method='under')
    pd.DataFrame(np.concatenate((Xs,np.expand_dims(ys,axis=1)),axis=1)).to_csv('../data/processed/mitbih_train_under.csv',index=None,header=None)

base sur echantillonée: N signaux par classe=13000

In [26]:
if not os.path.isfile('../data/processed/mitbih_train_smote_perturb_13000.csv'):
    Xo,yo = load_balanced.load_balanced_data(method='smote-perturb',n_normal=13000,
                       smote_perturb_smote_ratio=0.5)
    pd.DataFrame(np.concatenate((Xo,np.expand_dims(yo,axis=1)),axis=1)).to_csv('../data/processed/mitbih_train_smote_perturb_13000.csv',index=None,header=None)

calcul des features sur ces bases

In [29]:

if not os.path.isfile('../data/processed/features_mitbih_train_smote_perturb_13000.csv'):
    from features import build_features
    build_features.transform_inputs(files=['../data/processed/mitbih_train_smote_perturb_13000.csv'],
                     outdir = '../data/processed',
                    applyfilter='gaussian',
                      paramfilter=1,
                      Fs=125,
                      nech=6)

if not os.path.isfile('../data/processed/features_mitbih_train_under.csv'):
    from features import build_features
    build_features.transform_inputs(files=['../data/processed/mitbih_train_under.csv'],
                     outdir = '../data/processed',
                    applyfilter='gaussian',
                      paramfilter=1,
                      Fs=125,
                      nech=6)

## SVM sur features <a class="anchor" id="svm_features"></a>

In [50]:
from sklearn import preprocessing ,svm
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

### base sous echantillonée <a class="anchor" id="svm_features_sous"></a>

In [34]:
df = pd.read_csv('../data/processed/features_mitbih_train_under.csv')
df.head()
y_train = df.pop('target')
X_train = df

In [40]:
df = pd.read_csv('../data/processed/features_mitbih_test.csv')
y_test = df.pop('target')
X_test = df

In [35]:
df.describe()

,R,R_width,R_height,R_prominence,P1,P1_width,P1_height,P1_prominence,P2,P2_width,...,DN_OutlierInclude_n_001_mdrmd,SP_Summaries_welch_rect_area_5_1,SB_BinaryStats_diff_longstretch0,SB_MotifThree_quantile_hh,SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,SP_Summaries_welch_rect_centroid,FC_LocalSimple_mean3_stderr,DN_Mean,DN_Spread_Std
count,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000,...,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000,3205.000000
mean,0.689325,0.049522,0.880317,0.779519,0.278370,0.099335,0.439466,0.161242,0.540323,0.050653,...,0.456458,0.841568,10.989080,1.279279,0.597732,0.714646,0.124664,0.466039,0.193191,0.205874
std,0.257353,0.072996,0.226593,0.231055,0.106609,0.065709,0.257894,0.137754,0.225404,0.039998,...,0.303076,0.107027,4.237511,0.259850,0.306024,0.193863,0.117865,0.139453,0.106355,0.061075
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.957219,0.405671,3.000000,0.432294,0.146341,0.146341,0.024544,0.048135,0.014518,0.073758
25%,0.536000,0.022674,0.858855,0.712383,0.232000,0.058892,0.225527,0.036207,0.392000,0.027057,...,0.379679,0.783690,8.000000,1.095070,0.195122,0.731707,0.024544,0.353112,0.104051,0.166971
50%,0.712000,0.031576,0.981034,0.830056,0.272000,0.095765,0.369673,0.135317,0.512000,0.045183,...,0.534759,0.857924,11.000000,1.326754,0.804878,0.731707,0.098175,0.467978,0.169141,0.190748
75%,0.808000,0.053953,1.000000,0.944091,0.312000,0.121674,0.611429,0.265541,0.640000,0.065383,...,0.596257,0.929785,14.000000,1.422541,0.853659,0.853659,0.171806,0.564722,0.265729,0.231348
max,1.464000,0.757924,1.000000,1.000000,1.064000,0.989658,0.998090,0.872467,1.464000,0.525063,...,0.941176,0.990102,30.000000,2.154087,0.853659,0.853659,0.834486,0.983916,0.762032,0.400517


Application d'un scaler quand même

In [43]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled  = scaler.transform(X_train )
X_test_scaled  = scaler.transform(X_test )


#### premier test <a class="anchor" id="svm_features_sous_test"></a>

In [44]:
# Insérez votre code ici
clf = SVC(gamma=0.01,kernel='poly')
clf.fit(X_train_scaled,y_train)


SVC(gamma=0.01, kernel='poly')

In [49]:
y_pred = clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred))
pd.crosstab(y_test,y_pred,colnames=['prediction'])


              precision    recall  f1-score   support

         0.0       0.97      0.62      0.76     18118
         1.0       0.14      0.71      0.23       556
         2.0       0.90      0.67      0.77      1448
         3.0       0.03      0.96      0.06       162
         4.0       0.88      0.72      0.79      1608

    accuracy                           0.63     21892
   macro avg       0.59      0.74      0.52     21892
weighted avg       0.93      0.63      0.74     21892



prediction,0.0,1.0,2.0,3.0,4.0
target,,,,,
0.0,11214,2441,89,4232,142
1.0,99,395,4,58,0
2.0,114,28,971,322,13
3.0,2,2,2,156,0
4.0,91,2,9,346,1160


Ce n'est pas terrible :
* 306 malades non identifiés sur 3774 (8%)
* 6904 non malades considérés comme malades (38%)
* mauvais classement des malades

#### Gridsearch <a class="anchor" id="svm_features_sous_grid"></a>

Cherche de meilleurs hyper paramètres avec un grid search

In [86]:
from sklearn.metrics import make_scorer,recall_score
sc = make_scorer(recall_score,average = 'micro')

In [87]:
parametres = {'C':[0.1,1,10],'kernel': ['rbf', 'linear','poly'],'gamma':[0.001, 0.1, 0.5]}
grid_clf = GridSearchCV(clf,parametres,verbose=4,scoring=sc)
grid_clf = GridSearchCV(clf,parametres,verbose=4,scoring='f1_micro')


In [88]:
grid = grid_clf.fit(X_train_scaled, y_train)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END ....C=0.1, gamma=0.001, kernel=rbf;, score=0.711 total time=   1.7s
[CV 2/5] END ....C=0.1, gamma=0.001, kernel=rbf;, score=0.707 total time=   1.7s
[CV 3/5] END ....C=0.1, gamma=0.001, kernel=rbf;, score=0.735 total time=   1.7s
[CV 4/5] END ....C=0.1, gamma=0.001, kernel=rbf;, score=0.693 total time=   1.7s
[CV 5/5] END ....C=0.1, gamma=0.001, kernel=rbf;, score=0.702 total time=   1.7s
[CV 1/5] END .C=0.1, gamma=0.001, kernel=linear;, score=0.810 total time=   0.3s
[CV 2/5] END .C=0.1, gamma=0.001, kernel=linear;, score=0.807 total time=   0.3s
[CV 3/5] END .C=0.1, gamma=0.001, kernel=linear;, score=0.841 total time=   0.3s
[CV 4/5] END .C=0.1, gamma=0.001, kernel=linear;, score=0.833 total time=   0.3s
[CV 5/5] END .C=0.1, gamma=0.001, kernel=linear;, score=0.821 total time=   0.3s
[CV 1/5] END ...C=0.1, gamma=0.001, kernel=poly;, score=0.273 total time=   1.0s
[CV 2/5] END ...C=0.1, gamma=0.001, kernel=poly

In [90]:
grid.best_params_

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

In [91]:

y_pred = grid.best_estimator_.predict(X_test_scaled)
print(classification_report(y_test, y_pred))
pd.crosstab(y_test,y_pred,colnames=['prediction'])

              precision    recall  f1-score   support

         0.0       0.99      0.86      0.92     18118
         1.0       0.30      0.80      0.44       556
         2.0       0.54      0.94      0.69      1448
         3.0       0.21      0.90      0.34       162
         4.0       0.97      0.94      0.95      1608

    accuracy                           0.87     21892
   macro avg       0.60      0.89      0.67     21892
weighted avg       0.94      0.87      0.89     21892



prediction,0.0,1.0,2.0,3.0,4.0
target,,,,,
0.0,15545,1007,1032,489,45
1.0,63,444,41,5,3
2.0,23,10,1367,44,4
3.0,3,4,10,145,0
4.0,22,7,64,4,1511


In [102]:
b = grid.best_estimator_

In [108]:
parametres = {'C':[x for x in range(1,10)],'gamma':[0.1,0.2,0.3,0.4,0.5,0.6]}
grid_clf = GridSearchCV(clf,parametres,verbose=4,scoring='f1_micro')
grid = grid_clf.fit(X_train_scaled, y_train)


Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 1/5] END ....................C=1, gamma=0.1;, score=0.850 total time=   0.3s
[CV 2/5] END ....................C=1, gamma=0.1;, score=0.850 total time=   0.3s
[CV 3/5] END ....................C=1, gamma=0.1;, score=0.897 total time=   0.4s
[CV 4/5] END ....................C=1, gamma=0.1;, score=0.886 total time=   0.3s
[CV 5/5] END ....................C=1, gamma=0.1;, score=0.886 total time=   0.4s
[CV 1/5] END ....................C=1, gamma=0.2;, score=0.839 total time=   0.3s
[CV 2/5] END ....................C=1, gamma=0.2;, score=0.849 total time=   0.3s
[CV 3/5] END ....................C=1, gamma=0.2;, score=0.880 total time=   0.4s
[CV 4/5] END ....................C=1, gamma=0.2;, score=0.886 total time=   0.3s
[CV 5/5] END ....................C=1, gamma=0.2;, score=0.875 total time=   0.4s
[CV 1/5] END ....................C=1, gamma=0.3;, score=0.833 total time=   0.3s
[CV 2/5] END ....................C=1, gamma=0.3

In [111]:
grid.best_params_


{'C': 1, 'gamma': 0.1}

### Base sur echantillonée avec les paramètres du gridsearch obtenus sur le sous echantillonage (sinon c'est long )<a class="anchor" id="svm_features_sur"></a>


In [112]:
df = pd.read_csv('../data/processed/features_mitbih_train_smote_perturb_13000.csv')
df.head()
y_train = df.pop('target')
X_train = df

In [113]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled  = scaler.transform(X_train )
X_test_scaled  = scaler.transform(X_test )

In [114]:
clf = SVC(gamma=0.1,C=1,kernel='rbf')
clf.fit(X_train_scaled,y_train)

SVC(C=1, gamma=0.1)

In [115]:
y_pred = clf.predict(X_test_scaled)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98     18118
         1.0       0.54      0.85      0.66       556
         2.0       0.83      0.96      0.89      1448
         3.0       0.55      0.88      0.67       162
         4.0       0.98      0.98      0.98      1608

    accuracy                           0.96     21892
   macro avg       0.78      0.93      0.84     21892
weighted avg       0.97      0.96      0.96     21892



prediction,0.0,1.0,2.0,3.0,4.0
target,,,,,
0.0,17378,389,226,92,33
1.0,63,472,18,2,1
2.0,19,5,1395,25,4
3.0,8,1,10,143,0
4.0,7,5,27,0,1569


In [119]:
table = pd.crosstab(y_test,y_pred,colnames=['prediction'])
table

prediction,0.0,1.0,2.0,3.0,4.0
target,,,,,
0.0,17378,389,226,92,33
1.0,63,472,18,2,1
2.0,19,5,1395,25,4
3.0,8,1,10,143,0
4.0,7,5,27,0,1569


### Conclusions <a class="anchor" id="svm_features_sur_res"></a>

In [151]:
N_sains = table.loc[0,:].sum()
N_malades = table.sum().sum() - N_sain
print('total sains:'  ,N_sains)
print('total Malades:'  ,N_malades)
N_malades_non_detectes = table[0].iloc[1::].sum()

N_sains_mais_dits_malades = table.loc[0].iloc[1::].sum()
print('malades non détectés:' , N_malades_non_detectes , '(%.1f' % ( N_malades_non_detectes/N_malades*100) ,'%)')
print('sains envoyés en examen:' , N_sains_mais_dits_malades , '(%.1f' % (N_sains_mais_dits_malades/N_sains*100) ,'%)')

total sains: 18118
total Malades: 3774
malades non détectés: 97 (2.6 %)
sains envoyés en examen: 740 (4.1 %)



le seul problème est qu'il y a trop de  non malades positionnés en classe 1 mais c'est pas si mal

740 sains sont classés malades (4%)

In [160]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98     18118
         1.0       0.54      0.85      0.66       556
         2.0       0.83      0.96      0.89      1448
         3.0       0.55      0.88      0.67       162
         4.0       0.98      0.98      0.98      1608

    accuracy                           0.96     21892
   macro avg       0.78      0.93      0.84     21892
weighted avg       0.97      0.96      0.96     21892

